In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from scipy import signal
from rich.pretty import pprint


In [19]:
path = "./DREAMER.mat"
raw = sio.loadmat(path)

In [69]:
#Step 1 : frames
#from mne.time_frequency import psd_welch

def preprocess_EEG(raw, feature):

  frame = [[0 for x in range(640)] for y in range(12)] 

  #Dividing into frames
  for i in range(0,7680):
    j = i//640
    k = i%640
    frame[j][k] = raw[i]
  
  #frame[0][0-639] - Frame 1, frame[11][0-639] - Frame 12
  
  samplingRate = 128 
  nyquist = samplingRate/2.0
  n=213
  
  # delta 0-4 Hz Band Filter
  f = 4/nyquist
  f1 = 40/nyquist # EEG generally is in rage 1- 40 Hz. No interest is taken for freq. out of this range.
  
  fnew = [[0 for x in range(640)] for y in range(18)]
  pnew = [[0 for x in range(640)] for y in range(18)]
  
  thetamean =[]
  alphamean =[]
  betamean  =[]
  gammamean =[]
  deltamean =[]

  # delta_filter = signal.firwin(n, f, pass_zero=True, window ='hamming')
  overall_filter = signal.firwin(n, f1, pass_zero=True, window="hamming") #will be similar to original data

  for i in range(0,12):

    filt_overall = signal.filtfilt(overall_filter, 1, frame[i], method ='pad')
    # filt_delta = signal.filtfilt(delta_filter, 1, filt_overall)
    f,p = signal.welch(filt_overall,fs=128, nperseg=256)
    
    fnew[i] = f
    pnew[i] = p
    
    # print(f.shape) # (129,) why idk ?

    # plt.semilogy(fnew[i], pnew[i])
    # plt.xlabel('frequency [Hz]')
    # plt.ylabel('PSD')
    # plt.show()

    # c has frequency and reading together
    
    c =[]
    for k in range(0,129):
      x=[pnew[i][k],fnew[i][k]]
      c.append(x)

    # Taking mean of freq of one kind.

    count = 0.0; sum = 0.0; mean = 0.0
    for a in range(0,129):
      if(c[a][1] <=4.0):  #delta
        count +=1
        sum += c[a][0] 
    mean = sum/count
    deltamean.append(mean)

    count = 0.0; sum = 0.0; mean = 0.0
    for a in range(0,129):
      if(c[a][1] >4.0 and c[a][1] <=7.5 ): #theta
        count +=1
        sum += c[a][0] 
    mean = sum/count
    thetamean.append(mean)

    count = 0.0; sum = 0.0; mean = 0.0
    for a in range(0,129):
      if(c[a][1] >7.5 and c[a][1]<=13):   #alpha
        count +=1
        sum += c[a][0] 
    mean = sum/count
    alphamean.append(mean)

    count = 0.0; sum = 0.0; mean = 0.0
    for a in range(0,129):
      if(c[a][1] >13 and c[a][1]<=20):    #beta
        count +=1
        sum += c[a][0] 
    mean = sum/count
    betamean.append(mean)

    count = 0.0; sum = 0.0; mean = 0.0
    for a in range(0,129):
      if(c[a][1] >20 and c[a][1]<=40):    #gamma
        count +=1
        sum += c[a][0] 
    mean = sum/count
    gammamean.append(mean)
  
    fe=[]
    fe.append(deltamean[i])
    fe.append(thetamean[i])
    fe.append(alphamean[i])
    fe.append(betamean[i])
    fe.append(gammamean[i])
      
    feature.append(fe)

  return feature

In [70]:
def feat_extract_EEG(raw, electrode):
    EEG_tmp = np.zeros((23, 18, 60))
    for participant in range(0, 23):
    # for participant in range(0, 1):
        for video in range(0, 18):
        # for video in range(0, 1):

              B, S = [], []
                
              basl = (raw["DREAMER"][0, 0]["Data"][0, participant]["EEG"][0, 0]["baseline"][0, 0][video, 0][:, electrode])
              # Shape = (7808, )
              stim = (raw["DREAMER"][0, 0]["Data"][0, participant]["EEG"][0, 0]["stimuli"][0, 0][video, 0][:, electrode])
              # Shape = (large num 8k+, )
                
              B = preprocess_EEG(basl, B) # baseline psd received
              S = preprocess_EEG(stim, S) # stimulation psd received

              # B - list of shape (12, 5)
              # S - list of shape (12, 5)

              
              A = np.divide(S, B)   # relative density(absolute is diff for each individual)
              column = 0

              for k in range(0,5): # 5 - delta, theta, alpha, beta, gamma
                for l in range(0,12): # 12 -frames
                  EEG_tmp[participant, video,column] = A[l][k]
                  column+=1
                                

    col = []

    for i in range(0, 12):
      col.append("psddelta_"+str(i + 1)+"_un")
    for i in range(0, 12):
      col.append("psdtheta_"+str(i + 1)+"_un")
    for i in range(0, 12):
      col.append("psdalpha_"+str(i + 1)+"_un")
    for i in range(0, 12):
     col.append("psdbeta_"+str(i + 1)+"_un")
    for i in range(0, 12):
      col.append("psdgamma_"+str(i + 1)+"_un")

    print(EEG_tmp.shape)
    EEG_tmp = EEG_tmp.reshape(-1,EEG_tmp.shape[2])
    print(EEG_tmp.shape)
    data_EEG = pd.DataFrame(EEG_tmp, columns=col)
    
    for i in range(len(col)):
        data_EEG[col[i][:-3]] = data_EEG[[col[i]]]
        
    data_EEG.drop(col, axis=1, inplace=True)
    return data_EEG

In [71]:
feat_extract_EEG(raw, 1)

(23, 18, 60)
(414, 60)


,psddelta_1,psddelta_2,psddelta_3,psddelta_4,psddelta_5,psddelta_6,psddelta_7,psddelta_8,psddelta_9,psddelta_10,...,psdgamma_3,psdgamma_4,psdgamma_5,psdgamma_6,psdgamma_7,psdgamma_8,psdgamma_9,psdgamma_10,psdgamma_11,psdgamma_12
0,0.668270,0.471763,1.133646,0.612308,1.325317,0.797937,0.788123,0.463500,0.657070,0.566102,...,1.164679,1.402178,1.118319,1.038295,1.151707,1.222390,1.105916,1.042445,1.048702,1.318811
1,1.299305,64.323775,1.297122,0.832349,0.680891,1.821766,0.811991,27.106992,4.005335,1.097846,...,1.187209,1.110496,1.087197,1.074255,1.000368,0.932521,1.045339,1.172116,0.972757,1.128441
2,0.037661,1.695403,6.219202,1.201309,1.299122,0.982334,0.714569,1.382249,0.869015,1.084090,...,0.955623,1.025665,1.029439,1.083048,1.060751,0.933919,0.913906,0.908610,0.954304,1.067844
3,2.834254,1.553038,1.196003,1.531203,1.042025,1.806487,0.962725,1.593660,1.757192,1.288555,...,0.992479,0.977122,0.948230,1.032263,1.010007,0.893409,1.045856,1.004857,0.968264,0.961165
4,3.037603,6.425650,0.702872,0.863294,1.003678,0.952542,1.586216,34.889104,0.967640,1.449606,...,0.809810,0.837894,0.959477,0.945059,0.898079,0.762618,0.953387,0.826747,0.878203,0.863162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,0.579937,1.036997,2.166127,0.499652,1.134298,1.528229,1.397713,1.267686,0.976430,0.664316,...,1.215302,1.110987,1.038935,1.240229,1.183001,1.474007,0.847378,0.829736,0.702041,1.353282
410,9.988997,2.445418,1.048140,0.591348,4.576482,2.550255,1.026077,0.515773,0.849474,0.526932,...,0.487228,0.531820,0.682202,0.542898,0.444386,0.500234,0.449442,0.540423,0.840334,1.035036
411,14.815338,10.080441,1.329642,11.206073,2.182788,4.355111,3.299493,4.262765,0.947169,1.383544,...,2.201522,2.883159,0.410493,1.241163,1.131846,1.291274,0.904710,0.929907,1.237608,1.495616
412,0.270237,0.558681,1.364501,1.162670,2.874755,3.539256,2.856722,4.833029,1.445524,1.622411,...,0.850648,0.737923,0.643624,0.795712,1.137454,1.224007,0.666411,2.107676,0.688289,1.022300
